#### major /minor (as nc)

In [1]:
os.chdir(
#     "/Charles/project/ASm6A/shFUS/SNP_calling/08-Count/Hypothesis/Fisher/sig/"
#     "/Charles/project/ASm6A/shFUS/mapping/human/uniq_bam/count/Hypothesis/Fisher/sig/"
#     "/home/galaxy/project/alleleSpecific_analysis/data/shFUS/count/Hypothesis/Fisher/sig/"
    "/Charles/mjy/210505_A00869_0471_BH3NHMDSX2/count/Hypothesis/Fisher/sig/"
)
txt_list = [x for x in glob.glob("*.txt") if "overlap_" not in x]
result_dir = "matrix/forPlot/"
os.system("mkdir -p %s" % result_dir)


def format_file(in_file):
    prefix = os.path.basename(in_file).split(".txt")[0]
    df = pd.read_table(in_file, header=None)
    df.columns = [
        "contig", "position", "refAllele", "altAllele", "refRPKM_ratio",
        "altRPKM_ratio", "allelicRatio", "pvalue", "oddsratio", "qvalue",
        "refCount_ip", "altCount_ip", "refCount_input", "altCount_input",
        "mark"
    ]
    df = df[[
        'contig', 'position', 'refAllele', 'altAllele', "refRPKM_ratio","altRPKM_ratio",
        "mark"
    ]]
    df['refRPKM_ratio_%s'%prefix] = df["refRPKM_ratio"]
    df['altRPKM_ratio_%s'%prefix] = df['altRPKM_ratio']
    df['mark_%s' % prefix] = df['mark']
    del df['refRPKM_ratio']
    del df['altRPKM_ratio']
    del df['mark']
    return df


df = format_file(txt_list[0])
for txt in txt_list[1:]:
    df_i = format_file(txt)
    df = df.merge(df_i,
                  on=['contig', 'position', 'refAllele', 'altAllele'],
                  how="outer")
col_list = df.columns[4:]
col_list = sorted(col_list)
res = os.path.join(result_dir, "totalSamples_forPlot.txt")
new_cols = ['contig', 'position', 'refAllele', 'altAllele'] + col_list
df[new_cols].to_csv(res, sep="\t", index=False)

In [4]:
#### distinguish major and minor (the same as nc sample)
# os.chdir("/Charles/mjy/210505_A00869_0471_BH3NHMDSX2/count/reformat/overlapPeak/Hypothesis/Fisher/sig/matrix/forPlot/")
# os.chdir("/Charles/project/ASm6A/shFUS/mapping/human/uniq_bam/count/Hypothesis/Fisher/sig/matrix/forPlot/")
# os.chdir("/Charles/project/ASm6A/shFUS/SNP_calling/08-Count/Hypothesis/Fisher/sig/matrix/forPlot/")
os.chdir("/home/galaxy/project/alleleSpecific_analysis/data/shFUS/count/Hypothesis/Fisher/sig/matrix/forPlot/")
in_file = "totalSamples_forPlot.txt"

df = pd.read_table(in_file)
# wt_samples = ["wt1", "wt2"]
# kd_samples = ["kd1-1","kd1-2","kd2-1","kd2-2"]
# wt_samples = ['nc']
# kd_samples = ["T1","T2","T3"]
wt_samples = ["wt", "wt2"]
kd_samples = ["kd", "kd2"]
for wt_sample in wt_samples:
    wt_ref_ratio, wt_alt_ratio = 'refRPKM_ratio_%s'%wt_sample, 'altRPKM_ratio_%s'%wt_sample
    for kd_sample in kd_samples:
        kd_ref_ratio, kd_alt_ratio = 'refRPKM_ratio_%s'%kd_sample, 'altRPKM_ratio_%s'%kd_sample
        df_i = df[["contig",'position','refAllele','altAllele',
                   wt_ref_ratio, wt_alt_ratio,
                   kd_ref_ratio, kd_alt_ratio,
                   'mark_%s'%wt_sample, 'mark_%s'%kd_sample]]
        df_i = df_i.dropna()
        ### only ASm6As that were contained in wt samples were considered
        df_i = df_i[df_i['mark_%s'%wt_sample].isin(["ref","alt"])]
        ### at least one ASm6A ##################
#         mark_list = [x for x in df_i.columns if "mark" in x]
#         df_i['sum'] = (df_i[mark_list] == "unsig").sum(axis=1)
#         df_i = df_i[(df_i['sum'] <= 1)]
        #########################################
        df_1 = df_i[df_i[wt_ref_ratio] > df_i[wt_alt_ratio]]
        df_1['major_vs_minor_%s'%wt_sample] = df_1[wt_ref_ratio] /df_1[wt_alt_ratio]
        df_1['major_vs_minor_%s'%kd_sample] = df_1[kd_ref_ratio] /df_1[kd_alt_ratio]
        ###
        df_2 = df_i[df_i[wt_ref_ratio] <= df_i[wt_alt_ratio]]
        df_2['major_vs_minor_%s'%wt_sample] = df_2[wt_alt_ratio] /df_2[wt_ref_ratio]
        df_2['major_vs_minor_%s'%kd_sample] = df_2[kd_alt_ratio] /df_2[kd_ref_ratio]
        ###### merge
        df_m = pd.concat([df_1, df_2])
        ####
        res = "%s_vs_%s.bed"% (wt_sample, kd_sample)
        df_m['#contig'] = df_m['contig']
        df_m['start'] = df_m['position'] - 1
        df_m[['#contig','start','position','major_vs_minor_%s'%wt_sample,'major_vs_minor_%s'%kd_sample]].to_csv(res, sep="\t", index=False)

/home/galaxy/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/galaxy/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/galaxy/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/sta

In [7]:
####  overlap FUS or not
fus = "/Charles/project/ASm6A/data/RBP/union/ContainingFUS_genes_promoter.bed"
# fus = "/home/galaxy/Downloads/total/head2000_gene.bed"
# fus = "/home/galaxy/data/RBP/POSTAR/POSTAR_bed/FUS_HEK293T_gene.hg19.bed"
# fus = "/home/galaxy/data/RBP/POSTAR/POSTAR_bed/FUS_HEK293T_hg19.bed"

# os.chdir("/Charles/project/ASm6A/shFUS/SNP_calling/08-Count/Hypothesis/Fisher/sig/matrix/forPlot/")
# os.chdir("/Charles/mjy/210505_A00869_0471_BH3NHMDSX2/count/reformat/overlapPeak/Hypothesis/Fisher/sig/matrix/forPlot/")
# os.chdir("/Charles/project/ASm6A/shFUS/mapping/human/uniq_bam/count/Hypothesis/Fisher/sig/matrix/forPlot/")
os.chdir("/home/galaxy/project/alleleSpecific_analysis/data/shFUS/count/Hypothesis/Fisher/sig/matrix/forPlot/")
# result_dir = "overlap_FUS/"
result_dir = "None-targets/"
os.system("mkdir -p %s" % result_dir)
bed_list = glob.glob("*.bed")
for bed in bed_list:
    res = os.path.join(result_dir, bed)
#     os.system("bedtools intersect -a %s -b %s -wa | sort -u > %s" % (bed, fus, res))
    os.system("bedtools intersect -a %s -b %s -v -wa | sort -u > %s" % (bed, fus, res))

In [11]:
from scipy import stats
import matplotlib.pyplot as plt

#### calculate major/minor ratio
# os.chdir("/Charles/project/ASm6A/shFUS/SNP_calling/08-Count/Hypothesis/Fisher/sig/matrix/forPlot/overlap_FUS/") # overlap_FUS/
# os.chdir("/Charles/mjy/210505_A00869_0471_BH3NHMDSX2/count/reformat/overlapPeak/Hypothesis/Fisher/sig/matrix/forPlot/overlap_FUS/")
# os.chdir("/Charles/mjy/210505_A00869_0471_BH3NHMDSX2/count/reformat/overlapPeak/Hypothesis/Fisher/sig/matrix/forPlot/None-targets/")
os.chdir("/home/galaxy/project/alleleSpecific_analysis/data/shFUS/count/Hypothesis/Fisher/sig/matrix/forPlot/overlap_FUS/")

bed_list = glob.glob("*.bed")
for in_file in bed_list:
    wt, kd = in_file.split("_vs_")[0], in_file.split("_vs_")[1].split(".bed")[0]
#     in_file = "wt1_vs_kd1-1.bed"
    df = pd.read_table(in_file, sep="\t") # , header=None
    ## remove inf
    df = df[~df.isin([np.nan, np.inf, -np.inf]).any(1)]
    #mark_list = [x for x in df.columns if "major_vs_minor_" in x]
    #df = df[["contig", "position"] + mark_list]
    #kd_sample = 'major_vs_minor_T3'
    #df_1 = df[['major_vs_minor_nc', kd_sample]].dropna()

    # n_wt_list, n_kd_list = df_1['major_vs_minor_nc'].tolist(), df_1[kd_sample].tolist()
    n_wt_list, n_kd_list = df.iloc[:,3], df.iloc[:,4]
    # n_wt_list, n_kd_list = [abs(0.5-x) for x in n_wt_list], [abs(0.5-x) for x in n_kd_list]
    n_median = "wt:%s, kd:%s" % (str(round(np.median(n_wt_list),2)), str(round(np.median(n_kd_list),2)))
    n_stat, n_pvalue = stats.ttest_rel(n_wt_list, n_kd_list)
    print(wt, kd, len(n_wt_list), len(n_kd_list))

    ####
    x = [n_wt_list,n_kd_list]
    plt.boxplot(x, labels=["WT","FUS knockdown"])
    plt.ylabel('major/minor', size = 14)
    plt.xticks(rotation=0)
    plt.text(1.5, 2.5, 'pvalue=%e'%n_pvalue, horizontalalignment='center')
    plt.text(1.5, 3, n_median, horizontalalignment='center')
    plt.text(1.5, 3.5, "%s_vs_%s"% (wt, kd), horizontalalignment='center')
#     plt.show()
    ##
    plt.savefig(in_file.replace(".bed",".png"))
    plt.close()

wt2 kd2 20 20
wt kd2 21 21
wt kd 31 31
wt2 kd 26 26


In [39]:
os.chdir("/home/galaxy/project/alleleSpecific_analysis/data/shFUS/count/Hypothesis/Fisher/sig/matrix/forPlot/overlap_FUS/")
in_file = "wt2_vs_kd.bed"
wt, kd = in_file.split("_vs_")[0], in_file.split("_vs_")[1].split(".bed")[0]
#     in_file = "wt1_vs_kd1-1.bed"
df = pd.read_table(in_file, sep="\t", header=None) # 
## remove inf
df = df[~df.isin([np.nan, np.inf, -np.inf]).any(1)]
# print(df.head())
df = df.sample(n=11)
#mark_list = [x for x in df.columns if "major_vs_minor_" in x]
#df = df[["contig", "position"] + mark_list]
#kd_sample = 'major_vs_minor_T3'
#df_1 = df[['major_vs_minor_nc', kd_sample]].dropna()

# n_wt_list, n_kd_list = df_1['major_vs_minor_nc'].tolist(), df_1[kd_sample].tolist()
n_wt_list, n_kd_list = df.iloc[:,3], df.iloc[:,4]
# n_wt_list, n_kd_list = [abs(0.5-x) for x in n_wt_list], [abs(0.5-x) for x in n_kd_list]
n_median = "wt:%s, kd:%s" % (str(round(np.median(n_wt_list),2)), str(round(np.median(n_kd_list),2)))
n_stat, n_pvalue = stats.ttest_rel(n_wt_list, n_kd_list)
print(wt, kd, len(n_wt_list), len(n_kd_list))
print(n_stat, n_pvalue)

wt2 kd 11 11
1.6024424893118676 0.14014007199346554


In [19]:
#### overlap
in_file = "/Charles/project/ASm6A/shFUS/SNP_calling/08-Count/Hypothesis/Fisher/sig/matrix/forPlot/totalSamples_forPlot.txt"
df = pd.read_table(in_file)
wt_1, wt_2 = 'wt1', 'wt2'
kd_1, kd_2 = 'kd2-1', 'kd2-2'
wt_1_mark, wt_2_mark = 'mark_%s'%wt_1, 'mark_%s'%wt_2
kd_1_mark, kd_2_mark = 'mark_%s'%kd_1, 'mark_%s'%kd_2
df = df[(df[wt_1_mark] == df[wt_2_mark]) & (df[kd_1_mark] == df[kd_2_mark])]
df = df[~((df[wt_1_mark] == "unsig") & (df[kd_1_mark] == "unsig"))]
print(len(df))
# mark_cols = [x for x in df.columns if "mark" in x]
# df_mark = df[mark_cols]
# altRPKM_ratio_kd2-2
col_list = ['contig', 'position', 'refAllele', 'altAllele', 
            wt_1_mark, wt_2_mark, kd_1_mark, kd_2_mark,
           'refRPKM_ratio_%s'%wt_1, 'altRPKM_ratio_%s'%wt_1, 
           'refRPKM_ratio_%s'%wt_2, 'altRPKM_ratio_%s'%wt_2,
           'refRPKM_ratio_%s'%kd_1, 'altRPKM_ratio_%s'%kd_1,
           'refRPKM_ratio_%s'%kd_2, 'altRPKM_ratio_%s'%kd_2]
df = df[col_list]
# print(df)
### gain
df_gain = df[(df[wt_1_mark]=="unsig") & (df[kd_1_mark].isin(["ref","alt"]))]
print(df_gain)
df_gain['wt_score'] = 1
df_gain['kd_score'] = np.where(df[kd_1_mark]=="ref", 
                               ((df['refRPKM_ratio_%s'%kd_1]/df['altRPKM_ratio_%s'%kd_1]) + 
                               (df['refRPKM_ratio_%s'%kd_2]/df['altRPKM_ratio_%s'%kd_2])) /2, 
                               ((df['altRPKM_ratio_%s'%kd_1]/df['refRPKM_ratio_%s'%kd_1]) + 
                               (df['altRPKM_ratio_%s'%kd_2]/df['refRPKM_ratio_%s'%kd_2])) /2)
print(len(df_gain))
### loss
df_loss = df[(df[wt_1_mark].isin(["ref","alt"])) & (df[kd_1_mark]=="unsig")]
df_loss['kd_score'] = 1
df_loss['wt_score'] = np.where(df[wt_1_mark]=="ref", 
                               ((df['refRPKM_ratio_%s'%wt_1]/df['altRPKM_ratio_%s'%wt_1]) + 
                               (df['refRPKM_ratio_%s'%wt_2]/df['altRPKM_ratio_%s'%wt_2])) /2, 
                               ((df['altRPKM_ratio_%s'%wt_1]/df['refRPKM_ratio_%s'%wt_1]) + 
                               (df['altRPKM_ratio_%s'%wt_2]/df['refRPKM_ratio_%s'%wt_2])) /2)
print(len(df_loss))
### unchanging:
# ref --> ref
df_ref = df[(df[wt_1_mark]=="ref") & (df[kd_1_mark]=="ref")]
df_ref['wt_score'] = ((df['refRPKM_ratio_%s'%wt_1]/df['altRPKM_ratio_%s'%wt_1]) + 
                               (df['refRPKM_ratio_%s'%wt_2]/df['altRPKM_ratio_%s'%wt_2])) /2
df_ref['kd_score'] = ((df['refRPKM_ratio_%s'%kd_1]/df['altRPKM_ratio_%s'%kd_1]) + 
                               (df['refRPKM_ratio_%s'%kd_2]/df['altRPKM_ratio_%s'%kd_2])) /2
print(len(df_ref))
# alt --> alt
df_alt = df[(df[wt_1_mark]=="alt") & (df[kd_1_mark]=="alt")]
df_alt['wt_score'] = ((df['altRPKM_ratio_%s'%wt_1]/df['refRPKM_ratio_%s'%wt_1]) + 
                               (df['altRPKM_ratio_%s'%wt_2]/df['refRPKM_ratio_%s'%wt_2])) /2
df_alt['kd_score'] = ((df['altRPKM_ratio_%s'%kd_1]/df['refRPKM_ratio_%s'%kd_1]) + 
                               (df['altRPKM_ratio_%s'%kd_2]/df['refRPKM_ratio_%s'%kd_2])) /2
print(len(df_alt))
# unsig --> unsig
df_unsig = df[(df[wt_1_mark]=="unsig") & (df[kd_1_mark]=="unsig")]
print(len(df_unsig))
### reverse (should be removed)

31
    contig   position         ...         refRPKM_ratio_kd2-2 altRPKM_ratio_kd2-2
430   chr1   33402515         ...                   12.591837           33.324020
444   chr1  157095470         ...                   35.976678           14.165390
467   chr2   46986416         ...                   19.342300           33.987184
507   chr3  126170695         ...                   24.468009           10.444842
649  chr15   80886513         ...                   50.628470           24.951567
683  chr16   70415567         ...                   57.301564           22.599950
725  chr18   24035675         ...                   35.847729           58.099434
752  chr21   47552582         ...                    6.528026            2.836871

[8 rows x 16 columns]
8
12
6
4
0


/home/galaxy/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/galaxy/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/galaxy/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/sta